In [1]:
#Packages 
import numpy as np
import xarray as xr
import cmocean.cm as cmo
import matplotlib.pyplot as plt
import mpas_tools
from mpas_tools.planar_hex import make_planar_hex_mesh
from mpas_tools.mesh.conversion import convert, cull, mask
from mpas_tools.io import write_netcdf
from geometric_features import GeometricFeatures
from geometric_features import read_feature_collection
import json
import mosaic
from xhistogram.xarray import histogram

In [2]:
bmm_filepath = '/pscratch/sd/b/bmoorema/run_001_062/'
ds = xr.open_dataset(bmm_filepath + '20210421_sim7_CORE_60to30E2r2.mpaso.hist.am.timeSeriesStatsMonthly.0063-09-01.nc')

/tmp/ipykernel_205048/1359007595.py:2: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset(bmm_filepath + '20210421_sim7_CORE_60to30E2r2.mpaso.hist.am.timeSeriesStatsMonthly.0063-09-01.nc')


In [3]:
ds

<xarray.Dataset> Size: 4GB
Dimensions:                                                                                         (
                                                                                                     nMerHeatTransBinsP1: 181,
                                                                                                     nMocStreamfunctionBinsP1: 181,
                                                                                                     Time: 1,
                                                                                                     nCells: 236853,
                                                                                                     nVertLevels: 60,
                                                                                                     nEdges: 719506,
                                                                                                     nVertLevelsP1: 61,
                                                                                                     nRegions: 4,
                                                                                                     nOceanRegions: 7,
                                                                                                     nOceanRegionsTmp: 7)
Dimensions without coordinates: nMerHeatTransBinsP1, nMocStreamfunctionBinsP1,
                                Time, nCells, nVertLevels, nEdges,
                                nVertLevelsP1, nRegions, nOceanRegions,
                                nOceanRegionsTmp
Data variables: (12/437)
    binBoundaryMerHeatTrans                                                                         (nMerHeatTransBinsP1) float32 724B ...
    binBoundaryMocStreamfunction                                                                    (nMocStreamfunctionBinsP1) float32 724B ...
    timeMonthly_counter                                                                             (Time) int32 4B ...
    xtime_startMonthly                                                                              (Time) |S64 64B ...
    xtime_endMonthly                                                                                (Time) |S64 64B ...
    timeMonthly_avg_daysSinceStartOfSim                                                             (Time) timedelta64[ns] 8B ...
    ...                                                                                              ...
    timeMonthly_avg_activeTracersSurfaceFluxRunoff_temperatureSurfaceFluxRunoff                     (Time, nCells) float32 947kB ...
    timeMonthly_avg_activeTracersSurfaceFluxRunoff_salinitySurfaceFluxRunoff                        (Time, nCells) float32 947kB ...
    timeMonthly_avg_activeTracersSurfaceFluxRemoved_temperatureSurfaceFluxRemoved                   (Time, nCells) float32 947kB ...
    timeMonthly_avg_activeTracersSurfaceFluxRemoved_salinitySurfaceFluxRemoved                      (Time, nCells) float32 947kB ...
    timeMonthly_avg_nonLocalSurfaceTracerFlux_nonLocalTemperatureSurfaceFlux                        (Time, nCells) float32 947kB ...
    timeMonthly_avg_nonLocalSurfaceTracerFlux_nonLocalSalinitySurfaceFlux                           (Time, nCells) float32 947kB ...
Attributes: (12/722)
    on_a_sphere:                                                  YES
    sphere_radius:                                                6.371229e+06
    model_name:                                                   mpas
    core_name:                                                    ocean
    history:                                                      mpirun -n 6...
    source:                                                       MPAS
    ...                                                           ...
    config_AM_harmonicAnalysis_use_K2:                            YES
    config_AM_harmonicAnalysis_use_K1:                            YES
    config_AM_harmonicAnalysis_u

In [5]:
bmm_filepath = '/pscratch/sd/b/bmoorema/run_001_062/'
ds = xr.open_dataset(bmm_filepath + '20210421_sim7_CORE_60to30E2r2.mpaso.hist.am.timeSeriesStatsMonthly.0063-09-01.nc')
# list(ds.keys())
# extract tracer and boundary forcing? flux terms
ds_fluxes = ds[['timeMonthly_avg_activeTracers_temperature','timeMonthly_avg_activeTracers_salinity','timeMonthly_avg_shortWaveHeatFlux', 
                'timeMonthly_avg_longWaveHeatFluxUp','timeMonthly_avg_longWaveHeatFluxDown','timeMonthly_avg_latentHeatFlux','timeMonthly_avg_sensibleHeatFlux','timeMonthly_avg_seaIceHeatFlux',
                'timeMonthly_avg_evaporationFlux','timeMonthly_avg_rainFlux','timeMonthly_avg_snowFlux','timeMonthly_avg_riverRunoffFlux','timeMonthly_avg_iceRunoffFlux',
                'timeMonthly_avg_seaIceFreshWaterFlux','timeMonthly_avg_seaIceSalinityFlux','timeMonthly_avg_salinitySurfaceRestoringTendency']]

/tmp/ipykernel_70655/394089037.py:2: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset(bmm_filepath + '20210421_sim7_CORE_60to30E2r2.mpaso.hist.am.timeSeriesStatsMonthly.0063-09-01.nc')


In [8]:
temp_flux = ds_fluxes.timeMonthly_avg_activeTracers_temperature
temp_flux

<xarray.DataArray 'timeMonthly_avg_activeTracers_temperature' (Time: 1,
                                                               nCells: 236853,
                                                               nVertLevels: 60)> Size: 57MB
[14211180 values with dtype=float32]
Dimensions without coordinates: Time, nCells, nVertLevels
Attributes:
    long_name:  potential temperature
    units:      degrees Celsius

I'm not proud of this, but I stalked the github to find *some* kind of mesh file... It's a 30-60km (so low resolution) mesh from??? somewhere? Idk 
here's the [link](https://github.com/MPAS-Dev/MPAS-QuickViz/blob/8c78368d03a31d78923361ae6814c407fef3b045/ocean/AMOC/watermassanalysis/notebooks/AMOC_calculations.ipynb)

In [19]:
meshfile = '/global/cfs/projectdirs/e3sm/inputdata/ocn/mpas-o/EC30to60E2r2/ocean.EC30to60E2r2.210210.nc'
ds = xr.open_dataset(meshfile)
ds
lonCell = np.rad2deg(ds['lonCell'].values[cells])
latCell = np.rad2deg(ds['latCell'].values[cells])
latEdge = np.rad2deg(ds['latEdge'].values)
lonCell = np.where(lonCell > 180, lonCell - 360, lonCell)
area = ds.areaCell.values[cells]
depths = ds.refBottomDepth.values
dvEdge = ds.dvEdge.values
cellsOnEdge = ds.cellsOnEdge.values - 1

NameError: name 'cells' is not defined

In [14]:
ds['cellsOnEdge']

KeyError: "No variable named 'cellsOnEdge'. Variables on the dataset include ['binBoundaryMerHeatTrans', 'binBoundaryMocStreamfunction', 'timeMonthly_counter', 'xtime_startMonthly', 'xtime_endMonthly', ..., 'timeMonthly_avg_activeTracersSurfaceFluxRunoff_salinitySurfaceFluxRunoff', 'timeMonthly_avg_activeTracersSurfaceFluxRemoved_temperatureSurfaceFluxRemoved', 'timeMonthly_avg_activeTracersSurfaceFluxRemoved_salinitySurfaceFluxRemoved', 'timeMonthly_avg_nonLocalSurfaceTracerFlux_nonLocalTemperatureSurfaceFlux', 'timeMonthly_avg_nonLocalSurfaceTracerFlux_nonLocalSalinitySurfaceFlux']"